# Datos

In [ ]:
import pandas as pd
import requests
import json
import random
import datetime as dt
from matplotlib import pyplot as plt
import numpy as np
import os
from tqdm import tqdm
from collections import defaultdict, Counter
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from sklearn.model_selection import train_test_split
from mlxtend.frequent_patterns import apriori
import csv
import seaborn as sns

In [ ]:
!wget https://www.dropbox.com/scl/fi/js59yb4l6hiwasqazu4tv/train_data_lite.jl?rlkey=ka1tvky72q4re0ypsiefzqjja&st=2oqt9f9a&dl=0
!wget https://www.dropbox.com/scl/fi/boj6tbtur7p34jc56sh2m/test_data_lite.jl?rlkey=qymb3n0vuwo32k9karg2qo3zc&st=6yu9y7fj&dl=0
!wget https://www.dropbox.com/scl/fi/4ehzedjlurungpof791l3/item_data_lite.jl?rlkey=tazblazp9ru0izsklavsvr7iq&st=yza585io&dl=0
!wget https://www.dropbox.com/scl/fi/n2btmvyp7n4ccotatwts6/item_data.jl?rlkey=wfkfpe7mag65zwrc9pd04tl0a&st=hrz6zj8y&dl=0

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


--2024-12-04 21:15:08--  https://www.dropbox.com/scl/fi/js59yb4l6hiwasqazu4tv/train_data_lite.jl?rlkey=ka1tvky72q4re0ypsiefzqjja
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc98d328cd0ee57cd17a42c8038b.dl.dropboxusercontent.com/cd/0/inline/Cfq7A68LMtygQS336Cph_n1_CQuo8ZEgRAG8H6-xEl8p3uk28psVpzORJvBOxuidVeBtDytR_6A0xVkn1Au8KJ1cfUNRmMqov2NF343jzYMizSMclyDBLLc5EuhBdBZBVZT_Rkk8pp5XM1bOPFr5dciX/file# [following]
--2024-12-04 21:15:09--  https://uc98d328cd0ee57cd17a42c8038b.dl.dropboxusercontent.com/cd/0/inline/Cfq7A68LMtygQS336Cph_n1_CQuo8ZEgRAG8H6-xEl8p3uk28psVpzORJvBOxuidVeBtDytR_6A0xVkn1Au8KJ1cfUNRmMqov2NF343jzYMizSMclyDBLLc5EuhBdBZBVZT_Rkk8pp5XM1bOPFr5dciX/file
Resolving uc98d328cd0ee57cd17a42c8038b.dl.dropboxusercontent.com (uc98d328cd0ee57cd17a42c8038b.dl.dropboxusercontent.com)... 162.125.2.15, 

- *train_df:* Una fracción del dataset original de train.
- *test_df:* Una fracción del dataset original de test.
- *item_df:* Una fracción del dataset original de item.
- *item_df_full:* dataset original de item.

In [ ]:
train_data = []
with open("train_data_lite.jl?rlkey=ka1tvky72q4re0ypsiefzqjja", 'r') as file:
    for line in file:
        train_data.append(json.loads(line))

train_df = pd.DataFrame(train_data)

test_data = []
with open("test_data_lite.jl?rlkey=qymb3n0vuwo32k9karg2qo3zc", 'r') as file:
    for line in file:
        test_data.append(json.loads(line))

test_df = pd.DataFrame(test_data)

item_data = []
with open("item_data_lite.jl?rlkey=tazblazp9ru0izsklavsvr7iq", 'r') as file:
    for line in file:
        item_data.append(json.loads(line))

item_data_full = []
with open("item_data.jl?rlkey=wfkfpe7mag65zwrc9pd04tl0a", 'r') as file:
    for line in file:
        item_data_full.append(json.loads(line))

item_df = pd.DataFrame(item_data)
item_df_full = pd.DataFrame(item_data_full)


print(train_df.head())
print(test_df.head())
print(item_df.head())
print(item_df_full.head())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                                        user_history  item_bought
0  [{'event_info': 1786148, 'event_timestamp': '2...      1748830
1  [{'event_info': 643652, 'event_timestamp': '20...       228737
2  [{'event_info': 248595, 'event_timestamp': '20...      1909110
3  [{'event_info': 'RADIOBOSS', 'event_timestamp'...      1197370
4  [{'event_info': 'AMAZFIT BIP', 'event_timestam...      2049207
                                        user_history
0  [{'event_info': 1572239, 'event_timestamp': '2...
1  [{'event_info': 'PENTE PRA MAQUINA BRITANIA', ...
2  [{'event_info': 1986443, 'event_timestamp': '2...
3  [{'event_info': 'CAPACETE HELMET', 'event_time...
4  [{'event_info': 1398419, 'event_timestamp': '2...
   item_id                                              title  \
0    94208  Rompepiedra  Suplemento Alimenticio Auxiliar D...   
1  1685016  Calido Hoodie Chaleco Cierre Cruzado Para Homb...   
2   493220  Monitor De Presión Arterial De Brazo Bluetooth...   
3   539599      Tapetes Ma

### Analisis de datos

In [ ]:
event_types = []
for user in train_df['user_history']:
    for event in user:
        event_types.append(event['event_type'])

event_df = pd.DataFrame(event_types, columns=['event_type'])

# Contar los tipos de eventos
event_counts = event_df['event_type'].value_counts()
print(event_counts)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


event_type
search    438739
view      433744
Name: count, dtype: int64


In [ ]:
# Contar cuántas veces se compraron cada item
bought_counts = train_df['item_bought'].value_counts()
print(bought_counts.head())


item_bought
1587422    50
1361154    36
1803710    33
548905     29
1748830    27
Name: count, dtype: int64


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Convertir la columna price a float para análisis
item_df_full['price'] = pd.to_numeric(item_df['price'], errors='coerce')

# Estadísticas descriptivas de los precios
price_stats = item_df['price'].describe()
print(price_stats)


count       1089
unique       723
top       120.00
freq          14
Name: price, dtype: object


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
total_views = 0
total_searches = 0
total_histories = 0

# Recorrer el historial de cada usuario
for history in train_df['user_history']:
    view_count = 0
    search_count = 0

    # Contar "views" y "searches" en el historial
    for event in history:
        if event['event_type'] == 'view':
            view_count += 1
        elif event['event_type'] == 'search':
            search_count += 1

    # Actualizar contadores globales
    total_views += view_count
    total_searches += search_count
    total_histories += 1

# Calcular promedios
average_views = total_views / total_histories
average_searches = total_searches / total_histories

# Mostrar resultados
print(f'Promedio de items vistos por usuario: {average_views:.2f}')
print(f'Promedio de búsquedas por usuario: {average_searches:.2f}')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Promedio de items vistos por usuario: 14.44
Promedio de búsquedas por usuario: 14.88


# Implementación baselines

## Métricas

In [ ]:
def calculate_recall_at_k(recommendations, bought_item):
    return 1 if bought_item in recommendations else 0

def calculate_ndcg_at_k(recommendations, bought_item):
    if bought_item in recommendations:
        index = recommendations.index(bought_item)
        return 1 / np.log2(index + 2)
    return 0

def calculate_novelty(recommendations):
    # Evitar división por cero
    novelty = 0
    for item in recommendations:
        popularity = item_popularity.get(item, 1)
        novelty += 1 / np.log(1 + popularity)
    if len(recommendations) > 0:
        return novelty / len(recommendations)
    else:
      return 0

def calculate_diversity(recommendations, item_df):
    categories = []
    no_category = 0
    for item in recommendations:
        if item in item_df.index:
            categories.append(item_df.loc[item, 'category_id'])
        else:
            no_category += 1
    if not categories:
        return 0
    diversity = 1 - (len(set(categories)) / len(categories))
    return diversity

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 1. Random

In [ ]:
item_df = pd.DataFrame(item_data)

all_item_ids = item_df_full['item_id'].unique()

def get_random_recommendations(n_recommendations=5):
    return random.sample(list(all_item_ids), n_recommendations)

random_recommendations = get_random_recommendations()
print(f'Recomendaciones aleatorias: {random_recommendations}')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Recomendaciones aleatorias: [776393, 1097290, 1081124, 425488, 627414]


## 2. Most Popular

In [ ]:
item_popularity = Counter()

for session in train_data:
    bought_item = session['item_bought']
    item_popularity[bought_item] += 1

# Obtener los ítems más populares
most_popular_items = [item for item, count in item_popularity.most_common()]
def get_most_popular_recommendations(k):
    return most_popular_items[:k]

popular_recommendations = get_most_popular_recommendations(5)
print(f'Recomendaciones más populares: {popular_recommendations}')

Recomendaciones más populares: [1587422, 1748830, 716822, 1361154, 1906937]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def get_recommendations(model, k):
    if model == 'MostPopular':
        return get_most_popular_recommendations(k)
    elif model == 'Random':
        return get_random_recommendations(k)
    else:
        raise ValueError("Modelo no soportado")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Definir K
K = 10

# Inicializar métricas
metrics = {
    'MostPopular': {'recall': 0, 'novelty': 0, 'diversity': 0},
    'Random': {'recall': 0, 'novelty': 0, 'diversity': 0}
}

num_sessions = len(train_data)

for session in tqdm(train_data, desc="Evaluando modelos"):
    bought_item = session['item_bought']

    for model in ['MostPopular', 'Random']:
        recommendations = get_recommendations(model, K)

        # Calcular Recall@k
        recall = calculate_recall_at_k(recommendations, bought_item)
        metrics[model]['recall'] += recall

        # Calcular Novedad
        novelty = calculate_novelty(recommendations)
        metrics[model]['novelty'] += novelty

        # Calcular Diversidad
        diversity = calculate_diversity(recommendations, item_df_full)
        metrics[model]['diversity'] += diversity

# Promediar las métricas
for model in metrics:
    metrics[model]['recall'] /= num_sessions
    metrics[model]['novelty'] /= num_sessions
    metrics[model]['diversity'] /= num_sessions

# Mostrar resultados
for model in metrics:
    print(f"Modelo: {model}")
    print(f"  Recall@{K}: {metrics[model]['recall']:.4f}")
    print(f"  Novedad: {metrics[model]['novelty']:.4f}")
    print(f"  Diversidad: {metrics[model]['diversity']:.4f}\n")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Evaluando modelos:   1%|▏         | 129/10001 [00:20<26:01,  6.32it/s]


KeyboardInterrupt: 

## 3. GRU4Rec

In [ ]:
!git clone https://github.com/hidasib/GRU4Rec_PyTorch_Official --single-branch src

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Cloning into 'src'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 75 (delta 8), reused 4 (delta 4), pack-reused 62 (from 1)
Receiving objects: 100% (75/75), 364.46 KiB | 1.15 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [ ]:
def process(dst_path):
    # Cargar train_data
    os.makedirs(dst_path, exist_ok=True)
    data = []
    with open("train_data_lite.jl?rlkey=ka1tvky72q4re0ypsiefzqjja", 'r') as f:
        for line in f:
            data.append(json.loads(line))

    # Procesar las sesiones de usuarios, filtrando por 'view'
    sessions = []
    for session in data:
        views = [event for event in session['user_history'] if event['event_type'] == 'view']
        if len(views) > 1:  # Filtrar sesiones con más de 1 evento 'view'
            for view in views:
                sessions.append([session['item_bought'], view['event_info'], view['event_timestamp']])

    # Convertir en DataFrame
    df = pd.DataFrame(sessions, columns=['ItemBought', 'ItemId', 'TimeStr'])
    df['Time'] = df['TimeStr'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%f%z').timestamp())
    del(df['TimeStr'])

    # Definir límites para test y validación (2 días para test y 2 días para validación)
    tmax = df.Time.max()
    session_max_times = df.groupby('ItemBought').Time.max()

    # Test: últimos 10 días
    test_window = 10 * 86400  # 10 días en segundos
    session_test = session_max_times[session_max_times >= tmax-test_window].index
    test = df[np.in1d(df.ItemBought, session_test)]
    test = test[np.in1d(test.ItemId, df.ItemId)]
    tslength = test.groupby('ItemBought').size()
    test = test[np.in1d(test.ItemBought, tslength[tslength >= 2].index)]
    print('Test set\n\tEvents: {}\n\tSessions: {}\n\tItems: {}'.format(len(test), test.ItemBought.nunique(), test.ItemId.nunique()))
    test.rename(columns={'ItemBought': 'SessionId'}, inplace=True)
    test.to_csv(dst_path + '/test.tsv', sep='\t', index=False)

    # Validación: días anteriores al test
    valid_window = 5 * 86400  # 5 días para validación
    session_valid = session_max_times[(session_max_times < tmax-test_window) & (session_max_times >= tmax-test_window-valid_window)].index
    valid = df[np.in1d(df.ItemBought, session_valid)]
    valid = valid[np.in1d(valid.ItemId, df.ItemId)]
    tslength = valid.groupby('ItemBought').size()
    valid = valid[np.in1d(valid.ItemBought, tslength[tslength >= 2].index)]
    print('Validation set\n\tEvents: {}\n\tSessions: {}\n\tItems: {}'.format(len(valid), valid.ItemBought.nunique(), valid.ItemId.nunique()))
    valid.rename(columns={'ItemBought': 'SessionId'}, inplace=True)
    valid.to_csv(dst_path + '/validation.tsv', sep='\t', index=False)

    # Entrenamiento: el resto
    session_train = session_max_times[session_max_times < tmax-test_window-valid_window].index
    train_tr = df[np.in1d(df.ItemBought, session_train)]
    print('Train set\n\tEvents: {}\n\tSessions: {}\n\tItems: {}'.format(len(train_tr), train_tr.ItemBought.nunique(), train_tr.ItemId.nunique()))
    train_tr.rename(columns={'ItemBought': 'SessionId'}, inplace=True)
    train_tr.to_csv(dst_path + '/train.tsv', sep='\t', index=False)

process("data")



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Test set
	Events: 156484
	Sessions: 3540
	Items: 72232
Validation set
	Events: 67049
	Sessions: 2232
	Items: 33912
Train set
	Events: 207005
	Sessions: 9337
	Items: 98602


<ipython-input-7-3ef8afe85dbf>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_tr.rename(columns={'ItemBought': 'SessionId'}, inplace=True)


In [ ]:
def train_fraction(df, fraction, prefix="train", path="./data/"):
    length = len(df['ItemId'])
    first_session = df.iloc[length - length//fraction].SessionId
    df = df.loc[df['SessionId'] >= first_session]
    itemids = df['ItemId'].unique()
    n_items = len(itemids)

    print('Fractioned data set\n\tEvents: {}\n\tSessions: {}\n\tItems: {}'.format(len(df), df.SessionId.nunique(), df.ItemId.nunique()))
    filename = path + '{}_{}.tsv'.format(prefix, fraction)
    df.to_csv(filename, sep='\t', index=False)
    print("Saved as {}".format(filename))

fraction = 64
train = pd.read_csv("data/" + 'train.tsv', sep='\t', dtype={'ItemId': np.int64})
train_fraction(train, fraction)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Fractioned data set
	Events: 62948
	Sessions: 2906
	Items: 32855
Saved as ./data/train_64.tsv


- *test_gru4rec_df:* dataset de test generado en el process como un archivo .tsv para ser usado por Gru4Rec

In [ ]:
test_gru4rec_df = pd.read_csv('./data/test.tsv', sep='\t')

test_gru4rec_df.sort_values(by=['SessionId', 'Time'], inplace=True)
test_gru4rec_df.to_csv('./data/test.tsv', sep='\t', index=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
!python src/run.py ./data/train.tsv -t ./data/test.tsv -ps n_epochs=5 -d "cpu"

Traceback (most recent call last):
  File "/content/src/run.py", line 7, in <module>
    with open('metadata.json', 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: 'metadata.json'


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# NDCG

info: https://www.kaggle.com/code/pedrofraguas/meli-challenge#Analysis

In [ ]:
print(type(train_data))  # Esto debe ser <class 'pandas.core.frame.DataFrame'>

# Si es una lista o alguna otra estructura, conviértelo en DataFrame
if not isinstance(train_data, pd.DataFrame):
    train_data = pd.DataFrame(train_data)  # Convierte a DataFrame si es necesario


<class 'list'>


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


info de: https://www.kaggle.com/code/jsaguiar/meli-data-quick-start

In [ ]:
from datetime import datetime
train_df = pd.DataFrame(train_data)

def preprocess_hist(df):
    df['user_view'] = pd.Series(dtype='object')
    df['timestamps'] = pd.Series(dtype='object')
    df['user_search'] = pd.Series(dtype='object')
    df['search_timestamps'] = pd.Series(dtype='object')

    for i in df.index:
        lista_view = []
        lista_time = []
        lista_search = []
        lista_search_t = []

        for item in df.user_history[i]:
            if item['event_type'] == 'view':
                lista_view.append(int(item['event_info']))
                time_string = item['event_timestamp'].replace("T", " ").split('.')[0]
                timestamp = datetime.timestamp(datetime.strptime(time_string, '%Y-%m-%d %H:%M:%S'))
                lista_time.append(int(timestamp))

            if item['event_type'] == 'search':
                lista_search.append(item['event_info'])
                time_string = item['event_timestamp'].replace("T", " ").split('.')[0]
                timestamp = datetime.timestamp(datetime.strptime(time_string, '%Y-%m-%d %H:%M:%S'))
                lista_search_t.append(int(timestamp))
        df.at[i, 'user_view'] = lista_view
        df.at[i, 'timestamps'] = lista_time
        df.at[i, 'user_search'] = lista_search
        df.at[i, 'search_timestamps'] = lista_search_t

    return df

train_df = preprocess_hist(train_df)
train_df.drop('user_history', axis=1, inplace=True)
train_df.to_pickle('train.pickle')
train_df.head(4)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,item_bought,user_view,timestamps,user_search,search_timestamps
0,1748830,"[1786148, 1786148, 1615991, 1615991, 1615991, ...","[1571484342, 1571484357, 1571484446, 157148451...",[RELOGIO SMARTWATCH],[1571484367]
1,228737,"[643652, 1156086, 1943604, 206667, 1282813, 22...","[1570384973, 1570441567, 1570441673, 157047440...","[DESMAMADEIRA ELETRICA, DESMAMADEIRA ELETRICA,...","[1570441529, 1570441577, 1570441579, 157047440..."
2,1909110,"[248595, 248595]","[1569933963, 1569936110]",[],[]
3,1197370,"[505541, 505541, 505541, 505541, 505541, 12300...","[1570551846, 1570551956, 1570551975, 157055200...","[RADIOBOSS, RADIOBOSS, SOUND FORGE, SOUND FORG...","[1570551831, 1570573952, 1570717976, 157087445..."


item_bought: ID for the purchased item

user_view: List of IDs for each item that the user has seen in the last week

timestamps: List of timestamps for each visualization

user_search: List of search tokens in the last week

search_timestamps: List of timestamps for each search **texto en negrita**

In [ ]:
test_df = pd.DataFrame(test_data)
test_df = preprocess_hist(test_df)
test_df.drop('user_history', axis=1, inplace=True)
test_df.to_pickle('test.pickle')
test_df.head(2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,user_view,timestamps,user_search,search_timestamps
0,"[1572239, 1572239, 1572239, 1572239, 1572239, ...","[1569522707, 1569522904, 1569523055, 156952313...","[CINE BOX 4K, CINE BOX 4K, CINE BOX 4K, RECEPT...","[1569804622, 1569804639, 1569804650, 156980468..."
1,"[831243, 1125393, 831243, 943786, 943786, 9437...","[1570998446, 1570998473, 1570998512, 157099853...","[PENTE PRA MAQUINA BRITANIA, PENTE PRA MAQUINA...","[1570998101, 1570998219, 1570998346, 157099889..."


In [ ]:
item_df_full = pd.DataFrame(item_data)
item_df_full.to_pickle('products.pickle')
item_df_full.head(2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,item_id,title,domain_id,product_id,price,category_id,condition
0,94208,Rompepiedra Suplemento Alimenticio Auxiliar D...,MLM-SUPPLEMENTS,None,279.45,MLM167994,new
1,1685016,Calido Hoodie Chaleco Cierre Cruzado Para Homb...,MLM-VESTS,None,590.00,MLM194211,new


In [ ]:
# Aca se utilizo chatgpt para mejorar la función, igualmente la base se encuentra en: https://www.kaggle.com/code/jsaguiar/meli-data-quick-start
def preprocess_hist(df):
    df['user_view'] = pd.Series(dtype='object')
    df['timestamps'] = pd.Series(dtype='object')
    df['user_search'] = pd.Series(dtype='object')
    df['search_timestamps'] = pd.Series(dtype='object')

    for i in df.index:
        lista_view = []
        lista_time = []
        lista_search = []
        lista_search_t = []
        for item in df.user_history[i]:
            if item['event_type'] == 'view':
                lista_view.append(int(item['event_info']))
                time_string = item['event_timestamp'].replace("T", " ").split('.')[0]
                timestamp = datetime.timestamp(datetime.strptime(time_string, '%Y-%m-%d %H:%M:%S'))
                lista_time.append(int(timestamp))
            if item['event_type'] == 'search':
                lista_search.append(item['event_info'])
                time_string = item['event_timestamp'].replace("T", " ").split('.')[0]
                timestamp = datetime.timestamp(datetime.strptime(time_string, '%Y-%m-%d %H:%M:%S'))
                lista_search_t.append(int(timestamp))

        df.at[i, 'user_view'] = lista_view
        df.at[i, 'timestamps'] = lista_time
        df.at[i, 'user_search'] = lista_search
        df.at[i, 'search_timestamps'] = lista_search_t

    return df

train_data['item_bought'] = train_data['item_bought'].astype(int)

rename_dict = {
    'title': 'bought_title', 'domain_id': 'bought_domain_id', 'price': 'bought_price',
    'category_id': 'bought_category_id', 'condition': 'bought_condition'
}

item_df['condition'] = item_df['condition'].map({'new': 0, 'used': 1, 'not_specified': 2})
item_df.drop('product_id', axis=1, inplace=True)
train_data = preprocess_hist(train_data)
train_data = train_data.merge(item_df, left_on='item_bought', right_on='item_id', how='left')
train_data = train_data.drop('item_id', axis=1).rename(rename_dict, axis=1)
train_data.head(2)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,user_history,item_bought,user_view,timestamps,user_search,search_timestamps,bought_title,bought_domain_id,bought_price,bought_category_id,bought_condition
0,"[{'event_info': 1786148, 'event_timestamp': '2...",1748830,"[1786148, 1786148, 1615991, 1615991, 1615991, ...","[1571484342, 1571484357, 1571484446, 157148451...",[RELOGIO SMARTWATCH],[1571484367],Relógio Medidor Inteligente Pulso Freqüência C...,MLB-SMARTWATCHES,90.00,MLB135384,0.0
1,"[{'event_info': 643652, 'event_timestamp': '20...",228737,"[643652, 1156086, 1943604, 206667, 1282813, 22...","[1570384973, 1570441567, 1570441673, 157047440...","[DESMAMADEIRA ELETRICA, DESMAMADEIRA ELETRICA,...","[1570441529, 1570441577, 1570441579, 157047440...",Bomba Eletrica Tira Leite Materno Bivolt G-tech,MLB-MILK_EXTRACTORS,169.00,MLB264021,0.0


In [ ]:
# Función que mezcla un dataframe entregado con el data frame de items
def melt_views(df):
    # Seleccionamos las columnas relevantes
    views = df[['item_bought', 'user_view', 'timestamps']]

    # Restablecemos el índice
    views = views.reset_index(drop=True)

    # Desapilamos las listas de vistas y timestamps
    views = views.apply(pd.Series.explode, axis=0).reset_index(drop=True)

    # Filtramos las filas que no tienen vistas
    views = views[~views['user_view'].isna()]  # Eliminamos vistas vacías

    # Aseguramos que user_view es una columna con tipo pandas antes del merge
    views['user_view'] = views['user_view'].astype('int64')

    # Realizamos el merge con item_data
    item_data_df = pd.DataFrame(item_data)
    views = views.merge(item_data_df, left_on='user_view', right_on='item_id', how='left').drop('item_id', axis=1)

    # Retornamos el DataFrame final
    return views

# Llamamos a la función melt_views
views_data = melt_views(train_data)

# Mostramos las primeras filas
views_data.head(3)




/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,item_bought,user_view,timestamps,title,domain_id,product_id,price,category_id,condition
0,1748830,1786148,1571484342,Relógio Inteligente Smartwatch Gt08 Touch Scre...,MLB-SMARTWATCHES,None,119.99,MLB135384,new
1,1748830,1786148,1571484357,Relógio Inteligente Smartwatch Gt08 Touch Scre...,MLB-SMARTWATCHES,None,119.99,MLB135384,new
2,1748830,1615991,1571484446,Mochila Galáxia Compartimento P/ Laptop,MLB-SMARTWATCHES,None,79.71,MLB135384,new


## partida implementación

In [ ]:
# Crear el diccionario metadata
metadata = {}
for _, row in views_data.iterrows():
    metadata[row['user_view']] = {
        'domain_id': row['domain_id'],
        'price': row['price'],
        'category_id': row['category_id'],
        'condition': row['condition']
    }
    metadata[row['item_bought']] = {
        'domain_id': row['domain_id'],
        'price': row['price'],
        'category_id': row['category_id'],
        'condition': row['condition']
    }


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def ndcg(y_pred, y_true):
    dcg = 0
    idcg = 22.42461597 * len(y_true)
    contador = 0
    contador2 = 0
    contador3 = 0
    for pred, true in zip(y_pred, y_true):
        contador3 +=1
        position = 1
        for item in pred:
            if item == true:
                dcg += 12 / (np.log(1 + position))
                contador += 1
            elif item in metadata and true in metadata:
                if metadata[item]['domain_id'] == metadata[true]['domain_id']:
                    dcg += 1 / (np.log(1 + position))
                    contador2 +=1
            position += 1
    print(contador)
    print(contador2)
    print(contador3)

    score = dcg / idcg if idcg != 0 else 0
    return score

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from collections import Counter
item_popularity = Counter()
for bought_item in train_data['item_bought']:
    item_popularity[bought_item] += 1

most_popular_items = [item for item, count in item_popularity.most_common()]

def get_most_popular_recommendations(k):
    return most_popular_items[:k]

popular_recommendations = get_most_popular_recommendations(10)
print(f'Recomendaciones más populares: {popular_recommendations}')


Recomendaciones más populares: [1587422, 1361154, 1803710, 548905, 1748830, 716822, 1371799, 563256, 10243, 883507]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
y_true = train_data['item_bought'].tolist()

y_pred = []
for _ in range(len(test_df)):
    recommendations = get_most_popular_recommendations(10)
    y_pred.append(recommendations)
ndcg_score = ndcg(y_pred, y_true)
print(f'NDCG score: {ndcg_score}')

2
2
44
NDCG score: 2.168317518718226e-05


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
all_item_ids = item_df['item_id'].unique()
y_pred_random = []
def get_random_recommendations(n_recommendations=10):
    return random.sample(list(all_item_ids), n_recommendations)
for session in range(len(test_df)):
    recommendations = get_random_recommendations(10)
    y_pred_random.append(recommendations)

print(f'Recomendaciones aleatorias: {y_pred_random}')

ndcg_score_random = ndcg(y_pred_random, y_true)
print(f'NDCG score para el modelo aleatorio: {ndcg_score_random}')

Recomendaciones aleatorias: [[500760, 1072493, 1652941, 1813208, 181998, 1706875, 1499828, 947711, 731262, 314316], [821334, 832973, 139329, 1515122, 1154325, 1319122, 660797, 1821357, 836753, 2078196], [1347734, 321226, 1019254, 793356, 1736467, 1559925, 1072493, 683871, 1750835, 272347], [752267, 534248, 27205, 1383149, 1847204, 1889570, 1622283, 1215744, 426711, 1362768], [1645466, 371159, 588575, 443576, 2087594, 948537, 317507, 1822560, 898478, 1805107], [500760, 1078976, 1870504, 144569, 671264, 659610, 997882, 1515953, 1812287, 633992], [1409097, 775678, 1434049, 1855813, 752782, 1587271, 1065282, 1983673, 932572, 692182], [1920287, 1803252, 1276894, 1736467, 943786, 836753, 616894, 1631969, 1947339, 1587395], [1254439, 1269741, 400496, 371159, 1203834, 745652, 1091597, 437959, 210415, 946877], [1777682, 1173476, 1882157, 897309, 103485, 981242, 398156, 566246, 1323855, 1587624], [880794, 344342, 771206, 498690, 2007407, 1803252, 830658, 1443547, 691876, 1093148], [87559, 901189

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Ambos son muy bajos, lo cual era de esperar para estos modelos, ya que no son "significativos"

In [ ]:
# Guardar metadata en un archivo JSON
with open('metadata.json', 'w') as f:
    json.dump(metadata, f)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
with open('./data/item_popularity.json', 'w') as f:
    json.dump(item_popularity, f)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
!python src/run.py ./data/train.tsv -t ./data/test.tsv -ps n_epochs=5 -d "cpu"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Creating GRU4Rec model on device "cpu"
SET   n_epochs   TO   5   (type: <class 'int'>)
Loading training data...
Loading data from TAB separated file: ./data/train.tsv
Started training
The dataframe is not sorted by SessionId, sorting now
Data is sorted in 0.09
Created sample store with 4882 batches of samples (type=GPU)
Epoch1 --> loss: 5.633090 	(24.78s) 	[125.84 mb/s | 8054 e/s]
Epoch2 --> loss: 3.134717 	(42.66s) 	[73.09 mb/s | 4678 e/s]
Epoch3 --> loss: 1.840205 	(23.51s) 	[132.63 mb/s | 8489 e/s]
Epoch4 --> loss: 1.121445 	(42.72s) 	[72.99 mb/s | 4671 e/s]
Epoch5 --> loss: 0.755089 	(41.70s) 	[74.78 mb/s | 4786 e/s]
Total training time: 197.57s
Loading test data...
Loading data from TAB separated file: ./data/test.tsv
Starting evaluation (cut-off=[20], using standard mode for tiebreaking)
Using existing item ID map
The dataframe is already sorted by SessionId, Time
paso aca bien
Evaluation took 79.69s
Recall@20: 0.519849 MRR@20: 0.356368
45518
0
87560
NDCG Score: 0.304461629915917

### Modelo Híbrido

In [ ]:
def load_gru4rec_recommendations(file_path):
    """
    Carga el archivo CSV de recomendaciones de GRU4Rec.
    """
    df = pd.read_csv(file_path, sep=',', header=0, names=['session_id', 'predicted_items', 'actual_item'])
    # Convertir la columna 'predicted_items' de string a lista de enteros
    df['predicted_items'] = df['predicted_items'].apply(lambda x: list(map(int, str(x).strip("[]").split())))

    #eliminar la fila 0
    df = df.drop(0)

    return df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
gru4rec_file_path = "recommendations_gru4rec.csv"
gru4rec_df = load_gru4rec_recommendations(gru4rec_file_path)
print(gru4rec_df.head(5))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


   session_id                                    predicted_items  actual_item
1       70387  [42890, 42892, 42889, 62800, 62801, 42888, 215...        37762
2       10243  [105, 9413, 93, 9414, 90, 424, 107, 9939, 99, ...          833
3       98853  [59374, 59375, 59373, 59372, 59377, 59378, 451...        11284
4       21106  [16362, 16363, 97204, 97205, 16365, 97207, 972...        16362
5       34783  [36133, 59485, 59488, 59486, 59487, 26675, 594...        59488


In [ ]:
test_df2 = pd.read_csv('./data/test.tsv', sep='\t')

total_recall = 0
total_novelty = 0
total_diversity = 0
total_ndcg = 0
num_sessions = len(test_df2)

for session_id in test_df2['SessionId']:

  recommendations = gru4rec_df.loc[gru4rec_df['session_id'] == session_id, 'predicted_items'].values
  if recommendations.size > 0:
        recommendations = recommendations[0]

  # Calcular Novedad
  novelty = calculate_novelty(recommendations)
  total_novelty += novelty

  # Calcular Diversidad
  diversity = calculate_diversity(recommendations, item_df_full)
  total_diversity += diversity

recall = total_recall/num_sessions
novelty = total_novelty/num_sessions
diversity = total_diversity/num_sessions
ndcg = total_ndcg/num_sessions

print(f"Modelo: Gru4Rec")
print(f"  Novedad: {novelty:.4f}")
print(f"  Diversidad: {diversity:.4f}")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Modelo: Gru4Rec
  Novedad: 0.0711
  Diversidad: 0.0004


1. preprocesar train_data

In [ ]:
train_data_raw = []
with open("train_data_lite.jl?rlkey=ka1tvky72q4re0ypsiefzqjja", 'r') as file:
    for line in file:
        train_data_raw.append(json.loads(line))
search_sessions = []
for session in train_data_raw:
    search = [event for event in session['user_history'] if event['event_type'] == 'search']
    if len(search) > 1:  # Filtrar sesiones con más de 1 evento 'search'
        for view in search:
            search_sessions.append([session['item_bought'], view['event_info'], view['event_timestamp']])

print(len(search_sessions))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


436535


3. preprocesar item_data

In [ ]:
item_title_df = item_df_full[['item_id', 'title']].copy()
item_title_df = item_title_df[item_title_df['title'].notnull()]
#vemos el largo del df
item_title_df_lite = item_title_df[0:2000]
print(len(item_title_df_lite))


1089


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


4.Modelo de lenguaje

In [ ]:
!pip install sentence-transformers

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import re
from sentence_transformers import SentenceTransformer, util

# Preprocesamiento del texto
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Conservar números
    return text.strip()

# Cargar modelo SBERT
model = SentenceTransformer('all-MiniLM-L6-v2')

# Preprocesar términos de búsqueda y títulos de items
search_terms_df = pd.DataFrame(search_sessions, columns=['session_id', 'search_term', 'timestamp'])
search_terms_df['search_term'] = search_terms_df['search_term'].apply(preprocess_text)

item_df_full['title'] = item_df_full['title'].apply(preprocess_text)

# Generar embeddings
search_embeddings = model.encode(search_terms_df['search_term'].tolist(), convert_to_tensor=True)
item_embeddings = model.encode(item_df['title'].tolist(), convert_to_tensor=True)

# Calcular similitud de coseno
cosine_similarities = util.cos_sim(search_embeddings, item_embeddings).cpu().numpy()

# Crear un DataFrame de similitudes
similarity_df = pd.DataFrame(cosine_similarities, index=search_terms_df['session_id'], columns=item_df['item_id'])

# Mostrar algunas filas para verificar
print(similarity_df.head())


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

item_id      94208     1685016   493220    539599    1219935   216636   \
session_id                                                               
228737      0.236872  0.494632  0.062436  0.252531  0.297586  0.110424   
228737      0.236872  0.494632  0.062436  0.252531  0.297586  0.110424   
228737      0.236872  0.494632  0.062436  0.252531  0.297586  0.110424   
228737      0.236872  0.494632  0.062436  0.252531  0.297586  0.110424   
228737      0.236872  0.494632  0.062436  0.252531  0.297586  0.110424   

item_id      272347   1544437   221588    1130806  ...   1203834   1269268  \
session_id                                         ...                       
228737      0.317533  0.35942  0.106156  0.379276  ...  0.472371  0.069344   
228737      0.317533  0.35942  0.106156  0.379276  ...  0.472371  0.069344   
228737      0.317533  0.35942  0.106156  0.379276  ...  0.472371  0.069344   
228737      0.317534  0.35942  0.106156  0.379276  ...  0.472371  0.069344   
228737      0

5. Recomendación híbrida

In [ ]:
# from typing import final
# def get_hybrid_recommendations(session_id, gru4rec_df, similarity_df, alpha):
#     # Obtener recomendaciones del modelo GRU4Rec
#     if session_id in gru4rec_df.index:
#         gru_recommendations = gru4rec_df.loc[session_id, 'predicted_items']
#     else:
#         gru_recommendations = []

#     # Obtener similitudes basadas en búsqueda de texto
#     if session_id in similarity_df.index:
#         # Obtenemos la primera fila para el session_id, ya que hay duplicados
#         session_similarities = similarity_df.loc[session_id].iloc[0]
#         # Ordenamos las similitudes de mayor a menor y obtenemos los top 10 items
#         text_similarities = session_similarities.sort_values(ascending=False).head(10).index.tolist()
#     else:
#         text_similarities = []

#     # Reordenar las listas de gru_recomendations según el score de text similarity
#     gru_similarity = sorted(gru_recommendations, key=lambda x: text_similarities.index(x) if x in text_similarities else -1)

#     final_recommendations = gru_similarity

#     # Combinar ambas listas utilizando un enfoque híbrido con el parámetro alpha
#     combined_recommendations = list(set(gru_similarity + text_similarities))
#     print(combined_recommendations)
#     hybrid_scores = {}

#     for item in combined_recommendations:
#         print(gru4rec_df.head())
#         # Calcular puntajes híbridos usando los puntajes ajustados de GRU4Rec
#         gru_scores = {item: len(gru_recommendations) - idx for idx, item in enumerate(gru_recommendations)}
#         for item in combined_recommendations:
#             gru_score = gru_scores.get(item, 0)
#             text_score = similarity_df.loc[session_id].iloc[0].get(item, 0) if session_id in similarity_df.index else 0
#             hybrid_scores[item] = alpha * gru_score + (1 - alpha) * text_score

#         # #revisar si el session id está en similarity_df
#         # if session_id in similarity_df.index:
#         #     text_score = similarity_df.loc[session_id].iloc[0].get(item, 0)
#         # else:
#         #     text_score = 0
#         # # Calcular puntaje híbrido usando alpha
#         # hybrid_scores[item] = alpha * gru_score + (1 - alpha) * text_score

#     # Ordenar items por el puntaje híbrido y devolver los top 10
#     final_recommendations = sorted(hybrid_scores, key=hybrid_scores.get, reverse=True)[:10]

#     return final_recommendations

import matplotlib.pyplot as plt

def get_hybrid_recommendations(session_id, gru4rec_df, similarity_df, alpha):
    # Obtener recomendaciones del modelo GRU4Rec
    if session_id in gru4rec_df.index:
        gru_recommendations = gru4rec_df.loc[session_id, 'predicted_items']
    else:
        gru_recommendations = []

    # Crear diccionario de puntajes para GRU4Rec
    gru_scores = {item: len(gru_recommendations) - idx for idx, item in enumerate(gru_recommendations)}

    # Obtener similitudes basadas en búsqueda de texto
    if session_id in similarity_df.index:
        session_similarities = similarity_df.loc[session_id].iloc[0]
        text_similarities = session_similarities.sort_values(ascending=False).head(10)
        text_similarities_dict = text_similarities.to_dict()
        text_similarity_items = list(text_similarities.index)
    else:
        text_similarities_dict = {}
        text_similarity_items = []

    # Normalizar los puntajes de GRU4Rec
    if gru_scores:
        max_gru_score = max(gru_scores.values())
        min_gru_score = min(gru_scores.values())
        gru_scores = {item: (score - min_gru_score) / (max_gru_score - min_gru_score) if max_gru_score != min_gru_score else 0
                      for item, score in gru_scores.items()}

    # Normalizar los puntajes de similitud de texto
    if text_similarities_dict:
        max_text_score = max(text_similarities_dict.values())
        min_text_score = min(text_similarities_dict.values())
        text_similarities_dict = {item: (score - min_text_score) / (max_text_score - min_text_score) if max_text_score != min_text_score else 0
                                  for item, score in text_similarities_dict.items()}

    # Combinar ambas listas de recomendaciones en un conjunto único
    combined_recommendations = list(set(gru_recommendations + text_similarity_items))

    # Calcular puntajes híbridos para cada ítem combinado
    hybrid_scores = {}
    for item in combined_recommendations:
        gru_score = gru_scores.get(item, 0)  # Obtener puntaje de GRU
        text_score = text_similarities_dict.get(item, 0)  # Obtener puntaje de similitud textual
        # Calcular puntaje híbrido usando alpha
        hybrid_scores[item] = alpha * gru_score + (1 - alpha) * text_score

    # Ordenar ítems por puntaje híbrido y devolver los top 10
    final_recommendations = sorted(hybrid_scores, key=hybrid_scores.get, reverse=True)[:10]

    return final_recommendations


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Evaluación modelo híbrido

In [ ]:
test_df2 = pd.read_csv('./data/test.tsv', sep='\t')
y_pred_hybrid = []
for session_id in test_df2['SessionId']:
    recommendations = get_hybrid_recommendations(session_id, gru4rec_df, similarity_df, 0.5)
    y_pred_hybrid.append(recommendations)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
train_df2 = pd.read_csv('./data/train.tsv', sep='\t')
print(train_df2.head())
print(len(train_df2))

   SessionId   ItemId          Time
0     228737   643652  1.570399e+09
1     228737  1156086  1.570456e+09
2     228737  1943604  1.570456e+09
3     228737   206667  1.570489e+09
4     228737  1282813  1.570489e+09
207005


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# y_true = gru4rec_df['actual_item'].tolist()
# y_true = test_df2['SessionId'].tolist()
# print("YPred: ", len(y_pred_hybrid))
# print("YTrue: ", len( y_true))
# ndcg_score_hybrid = ndcg(y_pred_hybrid, y_true)
# print(f'NDCG score para el modelo híbrido: {ndcg_score_hybrid}')

total_recall = 0
total_novelty = 0
total_diversity = 0
total_ndcg = 0
num_sessions = len(test_df2)

for session_id in test_df2['SessionId']:

  recommendations = get_hybrid_recommendations(session_id, gru4rec_df, similarity_df, 0.5)

  # Calcular Recall@k
  recall = calculate_recall_at_k(recommendations, session_id)
  total_recall += recall

  # Calcular Novedad
  novelty = calculate_novelty(recommendations)
  total_novelty += novelty

  # Calcular Diversidad
  diversity = calculate_diversity(recommendations, item_df_full)
  total_diversity += diversity

  # Calcular NDCG
  ndcg = calculate_ndcg_at_k(recommendations, session_id)
  total_ndcg += ndcg

hybrid_recall = total_recall/num_sessions
hybrid_novelty = total_novelty/num_sessions
hybrid_diversity = total_diversity/num_sessions
hybrid_ndcg = total_ndcg/num_sessions

print(f"Modelo: Híbrido")
print(f"  Recall@10: {hybrid_recall:.4f}")
print(f"  Novedad: {hybrid_novelty:.4f}")
print(f"  Diversidad: {hybrid_diversity:.4f}")
print(f"  NDCG@10: {hybrid_ndcg:.4f}\n")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Modelo: Híbrido
  Recall@10: 0.0141
  Novedad: 1.2975
  Diversidad: 0.0000
  NDCG@10: 0.0075

